In [22]:
# import numpy as np
# import spacy
import os
import pandas as pd
# import re
# from collections import Counter
from math import log10
import spacy
import csv
# import nltk

# from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* ReadsDirectory: Reads Fake and Real folders and extracts the corresponding txt files(implementation done using os,walk function), tokenizes them and returns the corressponding class in lists.
* ReadsNews: Tokenization of words in txt files into a list 

In [4]:
# nlp = spacy.blank('ur')

def readNews(filename):
  # input: name of a the txt file output: list of words of the message in the email
  #data = [filename]
  data = []
  try:
    # content/NLP/Real/file1.txt
    fh = open(filename,'r',encoding = 'utf-8-sig')
  except IOError:
    print('cannot open', filename)
  # else:
    # reader = csv.reader(fh)
    # passage = list(reader)

  else:
    for line in fh:
      if line !='\n':
        words =  line[:-1].split(' ')
        for word in words:
          data.append(word)

    for line in fh:
      if line !='\n':
        words =  line[:-1].split(' ')
        # print("Filename = ",filename,"|| words  =",line)
        # break
        for word in words:
          data.append(word)
    # print("Passage  = ",passage)
    # passage.strip()
    # text = passage[0]
    # sptext = nlp (str(passage))
    # splist = []
    # print(sptext)
    # for word in sptext:
    #     data.append(str(word))
      # print(word)
    # print(data)
  finally:
    fh.close()
  # print("Filename = ",filename)
  # with open(filename,'r',encoding = 'utf-8-sig') as f:
  #   passage = f.read()
  #   # print("Passage = ",f)
  # f.close()
  # data = nltk.tokenize.wordpunct_tokenize(passage)
  # data = [re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,۔؟!''{}[] ‘'ٓ']", "", file) for file in data]

  # data = [x.replace(' ', '') for x in data]
  # r1 = ['،','%','!', '(',')',':','.',' ',"'",'"','“','۔','‘','’','؟','٪','\n','\n\n','\n\n\n']
  # while r1 in data: data.remove()
  # for replaced in r1:
  #   if replaced in data:
  #     print(r1)
      # data.remove(replaced)
  # if '\n' in data:
  #   data.remove('\n')
  # if '\n\n' in data:
  #   data.remove('\n\n')
  # if '\n\n\n' in data:
  #   data.remove('\n\n\n')
  # data = list(map(lambda x:x.strip(),data))
  # if '۔' in data:
  #   data.remove('۔')
  return data

def readDirectory(directory, class_name):
  # Takes the name of the directory and the name of the class and creates a subset of data
  d = list()
  # Directory = 
  # /content/NLP..../Real/file1.txt
  # /content/NLP..../Real/file2.txt
  for root, dirs, files in os.walk(directory):
    for name in files:
      d.append([readNews(os.path.join(root,name)),class_name])
      # print(d)
  return d

Sends directory of fake and real files and combines them into one data, they are distinguished by their classes, 1 is fake and 0 is real.  

In [5]:
#Read the data files without stop words
train_real_file  = '/content/drive/My Drive/NLP/Assignment5/Train/Real'
train_fake_file = '/content/drive/My Drive/NLP/Assignment5/Train/Fake'
test_real_file   = '/content/drive/My Drive/NLP/Assignment5/Test/Real'
test_fake_file  = '/content/drive/My Drive/NLP/Assignment5/Test/Fake'

# data_train = list()
data_train = readDirectory(train_fake_file,1) + readDirectory(train_real_file,0)
# print(len(data_train))
# r1 = ['،','%','!', '(',')',':','.',' ',"'",'"','“','۔','‘','’','؟','٪','\n','\n\n','\n\n\n']
# all_special = re.compile('|'.join(r1))
# for i in range(0,len(data_train)):
#   train_list = [re.sub(all_stops, '', word).strip() for word in data_train_1[i][0]]
#   data_train.append([new_list,data_train_1[i][1]])

# data_test = list()
data_test = readDirectory(test_fake_file,1) + readDirectory(test_real_file,0)
# for i in range(0,len(data_train)):
#   test_list = [re.sub(all_stops, '', word).strip() for word in data_test_1[i][0]]
#   data_test.append([new_list,data_test_1[i][1]])

# print(len(data_test))

These are the utility functions for Algorithm 1 and Algorithm 2.

In [6]:
def extractVocabulary(samples):
  vocabulary = []
  # "pakistan" if 90 times would be added once
  for sample in samples:
    #print (sample)
    for word in sample[0]:
      #print(word)
      if word not in vocabulary:
        vocabulary.append(word)
  return vocabulary

def countDocs(samples):
  return len(samples)

def countDocsInClass(samples,class_name): 
  # Data, "R"
  count = 0
  for sample in samples:
    if sample[1] == class_name:
      count += 1
  return count

def concatenateTextOfAllDocsInClass(samples,class_name):
  text_concatenated = []
  for sample in samples:
    if sample[1] == class_name:
      for word in sample[0]:
        text_concatenated.append(word)
  return text_concatenated

def countTokesnOfTerm(text,t):
  tokens = 0
  for word in text:
    if word == t:
      tokens += 1
  return tokens

def countWordsOfDocsInClass(samples,class_name):
  count = 0
  for sample in samples:
    if sample[1] == class_name:
      count = count + len(sample[0])
  return count



def extractTockensFromDoc(vocabulary,document_message):
  words_in_vocab = []
  for word in document_message:
    if word in vocabulary:
      words_in_vocab.append(word)
  return words_in_vocab

Code Implementation of Algorithm 1(trainMultinomialNB) and Algorithm 2(applymultinomialNB).

In [7]:
def trainMultinomialNB(C, D):
  V = extractVocabulary(D)
  
  B = len(V)
  N = countDocs(D)
  
  prior = pd.DataFrame(index = C ,columns = ['value_prior'])
  cond_prob = pd.DataFrame(index = C, columns = V)
  
  for c in C:
    # R
    Nc = countDocsInClass(D,c)
    prior['value_prior'][c] = Nc/N
    text_c = concatenateTextOfAllDocsInClass(D,c)
    num_words_tecxt_c = len(text_c)
    # print("For Class = ",c,"|| List  =",text_c )
    # print("For Class = ",c,"|| List  =",num_words_tecxt_c )
    for t in V:
      Tct = countTokesnOfTerm(text_c,t)
      cond_prob [t][c] = (Tct + 1)/(num_words_tecxt_c + B)
  return V,prior,cond_prob

def applyMultinomialNB(C,V,prior,cond_prob,d):
  W = extractTockensFromDoc(V,d)

  score = pd.DataFrame(index = C ,columns = ['value_score'])
  max_score = - 1000000000000.0
  result = -1
  
  for c in C:
    score['value_score'][c] = log10(prior['value_prior'][c])
    for t in W:
      score['value_score'][c] += log10(cond_prob[t][c])

    # Evaluate argmaxc∈Cscore[c]
    temp_score = score['value_score'][c]
    # print("Temp = ",temp_score)
    if temp_score > max_score:
      # print("Temp Max = ",temp_score)
      result = c
      max_score = temp_score
  # print("Result =", result)
  return result


Evaluation functions accuracy, precision,recall and f1

In [16]:
def accuracy(C,V,prior,cond_prob,dataset_test):
  # print(len(V))
  count_correct = 0
  for sample in dataset_test:
    temp = applyMultinomialNB(C,V,prior,cond_prob,sample[0])
    # print(temp, sample[1])
    if temp == sample[1]:
      count_correct += 1
  return float(count_correct)/len(dataset_test)

def precision(C,V,prior,cond_prob,dataset_test):
  # print(len(V))
  count_correct = 0
  false_pos = 0
  true_pos = 0
  for sample in dataset_test:
    temp = applyMultinomialNB(C,V,prior,cond_prob,sample[0])
    # print(temp, sample[1])
    if temp == sample[1] and temp == 0:
      true_pos += 1
    elif temp != sample[1] and temp == 0:
      false_pos += 1

  return float(true_pos)/(false_pos + true_pos)

def recall(C,V,prior,cond_prob,dataset_test):
  # print(len(V))
  count_correct = 0
  false_neg = 0
  true_pos = 0
  for sample in dataset_test:
    temp = applyMultinomialNB(C,V,prior,cond_prob,sample[0])
    # print(temp, sample[1])
    if temp == sample[1] and temp == 0:
      true_pos += 1
    elif temp != sample[1] and temp == 1:
      false_neg += 1

  return float(true_pos)/(false_neg + true_pos)

def fone(precision,recall):
  num = precision * recall
  den = precision + recall
  return 2 * float(num)/den

Training the data using Algorithm 1

In [10]:
C = [0,1]
V,prior,cond_prob = trainMultinomialNB(C, data_train)

In [17]:
print("Accuracy: ",accuracy(C,V,prior,cond_prob,data_test))
Pr = precision(C,V,prior,cond_prob,data_test)
print("Precision: ",Pr)
Rec = recall(C,V,prior,cond_prob,data_test)
print("Recall: ",Rec)
print("F1: ",fone(Pr,Rec))

Accuracy:  0.7748091603053435
Precision:  0.7935483870967742
Recall:  0.82
F1:  0.8065573770491803


Read set of stopwords

In [18]:
stops = list()
with open('/content/drive/My Drive/NLP/Assignment5/stopwords-ur.txt', mode='r', encoding='utf-8-sig') as f:
  # main = f.read()
  # line = main.strip().split()
  # line =line.lower()
  stops =  f.read().split()
  # uni = Counter(re.findall(r'\w+', main.lower()))
  # line = line.lower()
  # line = line.split(" ")
f.close()

print(stops)

['آئی', 'آئے', 'آج', 'آخر', 'آخرکبر', 'آدهی', 'آًب', 'آٹھ', 'آیب', 'اة', 'اخبزت', 'اختتبم', 'ادھر', 'ارد', 'اردگرد', 'ارکبى', 'اش', 'اضتعوبل', 'اضتعوبلات', 'اضطرذ', 'اضکب', 'اضکی', 'اضکے', 'اطراف', 'اغیب', 'افراد', 'الگ', 'اور', 'اوًچب', 'اوًچبئی', 'اوًچی', 'اوًچے', 'اى', 'اً', 'اًذر', 'اًہیں', 'اٹھبًب', 'اپٌب', 'اپٌے', 'اچھب', 'اچھی', 'اچھے', 'اکثر', 'اکٹھب', 'اکٹھی', 'اکٹھے', 'اکیلا', 'اکیلی', 'اکیلے', 'اگرچہ', 'اہن', 'ایطے', 'ایک', 'ب', 'ت', 'تبزٍ', 'تت', 'تر', 'ترتیت', 'تریي', 'تعذاد', 'تن', 'تو', 'توبم', 'توہی', 'توہیں', 'تٌہب', 'تک', 'تھب', 'تھوڑا', 'تھوڑی', 'تھوڑے', 'تھی', 'تھے', 'تیي', 'ثب', 'ثبئیں', 'ثبترتیت', 'ثبری', 'ثبرے', 'ثبعث', 'ثبلا', 'ثبلترتیت', 'ثبہر', 'ثدبئے', 'ثرآں', 'ثراں', 'ثرش', 'ثعذ', 'ثغیر', 'ثلٌذ', 'ثلٌذوثبلا', 'ثلکہ', 'ثي', 'ثٌب', 'ثٌبرہب', 'ثٌبرہی', 'ثٌبرہے', 'ثٌبًب', 'ثٌذ', 'ثٌذکرو', 'ثٌذکرًب', 'ثٌذی', 'ثڑا', 'ثڑوں', 'ثڑی', 'ثڑے', 'ثھر', 'ثھرا', 'ثھراہوا', 'ثھرپور', 'ثھی', 'ثہت', 'ثہتر', 'ثہتری', 'ثہتریي', 'ثیچ', 'ج', 'خب', 'خبرہب', 'خبرہی', 'خبرہے', 'خبهوظ

Stop Word removal

In [19]:
# def rem(text,t):
#   tokens = 0
#   for word in text:
#     if word == t:
#       tokens += 1
#   return tokens
def removalofStopWords(samples):
  new_sample = list()
  print(len(samples))
  # stopsw = 0
  # all_stops = re.compile('|'.join(stops))
  for i in range(0,len(samples)):
    # list(set(stops) & set(sample[i][0]))
    new_list = list(set(samples[i][0]) - set(stops))
    new_list = list(filter(None, new_list))
    # new_list = [i for i in samples[i][0] if i not in stops]
    # new_list = [re.sub(all_stops, '', word).strip() for word in samples[i][0]]

    # for word in samples[i][0]:
    #   new_list = re.sub(all_stops, '', word)
    # for word in list(samples[i][0]):
    #   if word in stops:
    #     print('in stops')
    #     samples[i][0].remove(word)
    # for word in samples[i][0]:
      # print(words)
      # if word in stops:
      #   print("stopword=",word)
      #   stopsw = stopsw + 1
      #   print("before = ",samples[i][0])
      #   samples[i][0].remove(word)
      #   print("after = ",samples[i][0])
        
    new_sample.append([new_list,samples[i][1]])
      #   break
  # print(concatenateTextOfAllDocsInClass(samples,0))
  # print(stopsw)
  return new_sample
stop_test = removalofStopWords(data_test)
# print(stops)
# print(stop_test)
# print(data_test)
# print(len(stop_test),len(data_test))
# print(len(extractVocabulary(stop_test)),len(extractVocabulary(data_test)))

# data_test[0][1]
# print(data_test,len(data_test))

262
[[['فوری', 'کا', 'معلومات', 'اقدام', 'مسائل', 'شفافیت', 'سراہتے', 'کرنے', 'امکان', 'میں', 'سے', 'شہزاد', 'جیسے', 'کوریج', 'پیدا', 'رائٹس', 'یکساں', 'حل', 'انتہائی', 'پراپرٹی', 'بڑا', 'ون', 'ڈیٹا', 'برادری', 'کاروباری', 'نائب', 'ناصر', 'آسانی', 'حصول', 'لاہور(کامرس', 'سب', 'ونڈو', 'خواجہ', 'آپریشن', 'مضبوطی', 'حیدر', 'مشکور', 'وہ', 'مسئلہ', 'ں', 'جغرافیائی', 'رپورٹر)', 'سسٹم', 'زمینی', 'الیکٹرونک', 'بہت', 'اضافے', 'چیمبر', 'حکومتی', 'گزار', 'تنازعات', 'بیس', 'عمل', 'کاروبار', 'کر', 'اقدامات', 'حکومت', 'الماس', 'لاہور', 'حدود', 'معلوم', 'انھو', 'رجسٹریشن', 'ٹیکنالوجی', 'ذریعے', 'کو', 'دیا', 'شکر', 'پاکستان', 'صدر', 'تھا', 'کہا', 'رسائی', 'انفراسٹرکچر', 'ٹھوس', 'جدید', 'زور', 'نے', 'سینئر'], 1], [['ساتھ', 'افتتاح', 'کا', 'بھی', 'اشیا', 'ٹیکنالوجی', 'صوبہ', 'ممالک', 'کو', 'انٹرنیٹ', 'مطابق', 'این', 'رپورٹ', 'اہتمام', 'یوان', 'انفارمیشن', '2017', 'جس', 'والی', 'سائنس', 'فیصد', '25', 'موجودہ', 'ڈیجیٹل', 'پی', 'کرنے', 'پاکستان', 'عہد', 'موضوع', 'مالیت', 'چینی', 'اضافہ', '(آئی', 'عوامی'

Checking Accuracy, Precision, Recall and F1 after Stop Word Removal

In [21]:
print("After removing stop Words")
print("Accuracy: ",accuracy(C,V,prior,cond_prob,stop_test))
Pr = precision(C,V,prior,cond_prob,stop_test)
print("Precision: ",Pr)
Rec = recall(C,V,prior,cond_prob,stop_test)
print("Recall: ",Rec)
print("F1: ",fone(Pr,Rec))

After removing stop Words
Accuracy:  0.7480916030534351
Precision:  0.7560975609756098
Recall:  0.8266666666666667
F1:  0.7898089171974522
